In [ ]:
# The '!' allows us to run terminal commands directly from the notebook.
!pip install ultralytics

In [ ]:
import yaml
import cv2
import random
import os
import matplotlib.pyplot as plt

# --- IMPORTANT: UPDATE THIS PATH ---
# You must replace this path with the full path to your downloaded dataset's data.yaml file.
yaml_file_path = # --- IMPORTANT: THIS IS THE CORRECT PATH FOR YOUR SETUP ---
yaml_file_path = r'C:\Users\Sreeneel Chavidi\OneDrive\Desktop\.vs code\.vs code\DNA-Proj\Project_dna\dataset-german\data.yaml'

# Load the data configuration file
with open(yaml_file_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

# Get paths and class names from the yaml file
# Note: Roboflow datasets often use relative paths. We might need to adjust them.
dataset_root = os.path.dirname(yaml_file_path)
train_images_path = os.path.join(dataset_root, data_yaml['train'])
class_names = data_yaml['names']

print(f"Dataset contains {len(class_names)} classes.")
print("Class Names:", class_names)

# --- Visualize a few random training images with their bounding boxes ---
image_files = [f for f in os.listdir(train_images_path) if f.endswith('.jpg')]

plt.figure(figsize=(15, 10))
for i in range(4): # Show 4 random images
    # Select a random image
    random_image_name = random.choice(image_files)
    image_path = os.path.join(train_images_path, random_image_name)
    label_path = image_path.replace('.jpg', '.txt').replace('images', 'labels')
    
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert for matplotlib
    h, w, _ = image.shape
    
    # Read the corresponding label file
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id, x_center, y_center, width, height = map(float, line.split())
                
                # Convert YOLO format to OpenCV format
                x1 = int((x_center - width/2) * w)
                y1 = int((y_center - height/2) * h)
                x2 = int((x_center + width/2) * w)
                y2 = int((y_center + height/2) * h)
                
                # Draw the bounding box and label
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, class_names[int(class_id)], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    ax = plt.subplot(2, 2, i + 1)
    plt.imshow(image)
    plt.title(random_image_name)
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8n model
model = YOLO('yolov8n.pt')

# Train the model using the dataset from the .yaml file
# The output will be saved to a new 'runs' directory
results = model.train(data=yaml_file_path, epochs=25, imgsz=640)

In [ ]:
from IPython.display import Image

# Display the results graph, which shows loss curves, mAP, and other metrics.
# The path to results.png may vary depending on the run number (train, train2, etc.)
Image(filename='runs/detect/train/results.png')

In [ ]:
from ultralytics import YOLO
import random
import os
from IPython.display import display

# Load our custom-trained model
model = YOLO('runs/detect/train/weights/best.pt')

# Get a list of validation images
val_images_path = os.path.join(dataset_root, data_yaml['val'])
val_image_files = [os.path.join(val_images_path, f) for f in os.listdir(val_images_path) if f.endswith('.jpg')]

# Run inference on a few random validation images
results = model(random.sample(val_image_files, 2)) # Predict on 2 random images

# Display the results
for r in results:
    # r.plot() returns a BGR numpy array of predictions
    # We can use IPython's Image to display it directly after converting BGR to RGB
    display(Image(data=cv2.imencode('.png', r.plot()[..., ::-1])[1].tobytes()))